# Subject Recognition CRF Tagger Model

In [1]:
import sys
sys.path.insert(0, '../../../allennlp')
sys.path.insert(0, '../../')

In [2]:
import os
from allennlp.common.params import Params

crf_tagger = {
    "dataset_reader": {
        "type": "sequence_tagging",
        "word_tag_delimiter": "/",
        "token_indexers": {
            "tokens": {
                "type": "single_id",
                "lowercase_tokens": True
            },
            "token_characters": {
                "type": "characters",
                "character_tokenizer": {
                    "end_tokens": ["@@PADDING@@", "@@PADDING@@", "@@PADDING@@", "@@PADDING@@"]
                }
            }
        }
    },
    "pytorch_seed": 4006490763, # 0.957081 Other random seeds ranged from 0.954 - 0.956
    "numpy_seed": 4006490763,
    "random_seed": 4006490763,
    # TODO: Update this to the location for subject name recognition training data
    "train_data_path": './../../data/subject_recognition/train.txt',
    "validation_data_path": './../../data/subject_recognition/dev.txt',
    "model": {
        "type": "crf_tagger",
        "text_field_embedder": {
            "tokens": {
                "type":
                    "embedding",
                "pretrained_file":
                    "https://s3-us-west-2.amazonaws.com/allennlp/datasets/glove/glove.6B.100d.txt.gz",
                "embedding_dim":
                    100,
                "trainable":
                    False
            },
            "token_characters": {
                "type": "character_encoding",
                "embedding": {
                    "embedding_dim": 25
                },
                "encoder": {
                    "type": "cnn",
                    "embedding_dim": 25,
                    "num_filters": 100,
                    "ngram_filter_sizes": [5]
                },
                "dropout": 0.25
            }
        },
        "encoder": {
            "type": "lstm",
            "input_size": 200,
            "hidden_size": 600,
            "num_layers": 3,
            "dropout": 0.25,
            "bidirectional": True
        },
        "regularizer": [["transitions$", {"type": "l2", "alpha": 0.01}]],
    },
    "iterator": {
        "type": "bucket",
        "sorting_keys": [["tokens", "num_tokens"]],
        "padding_noise": 0.3,
        "batch_size": 64,
    },
    "trainer": {
        "num_epochs": 100,
        "cuda_device": 0,
        "patience": 6,
        "optimizer": {
            "type": "adam",
            "amsgrad": False,
        },
        "validation_metric": "+accuracy",
        "learning_rate_scheduler":  {
          "type": "reduce_on_plateau",
          "factor": 0.5,
          "mode": "max",
          "patience": 2,
        },
    }
}

## Single Run

In [3]:
# from lib.utils import config_logging
# from lib.utils import new_experiment_folder
# from allennlp.commands.train import train_model

# config_logging()
# experiment_folder = new_experiment_folder(label='subject_recognition', parent_directory='../../experiments/')
# # experiment_folder = '../../experiments/subject_recognition.02_11_08:21:07'
# params = Params(crf_tagger)
# print('Serialization Directory:', experiment_folder)
# train_model(params=params, serialization_dir=experiment_folder)

## Grid Search

In [4]:
from copy import deepcopy
import itertools

token_characters_ = []
# base_rnn_token_characters = {
#     "type": "character_encoding",
#     "embedding": {
#         "embedding_dim": 25
#     },
#     "encoder": {
#         "type": "gru",
#         "input_size": 25,
#         "hidden_size": 50,
#         "num_layers": 2,
#         "dropout": 0.25,
#         "bidirectional": True
#     },
#     "dropout": 0.25,
# }
# rnn_token_characters_space = [[0.0, 0.25, 0.5], [1, 2, 3], ['gru', 'lstm']]
# for dropout, num_layers, type_ in list(itertools.product(*rnn_token_characters_space)):
#     copy = deepcopy(base_rnn_token_characters)
#     copy['encoder']['dropout'] = dropout
#     copy['dropout'] = dropout
#     copy['encoder']['num_layers'] = num_layers
#     copy['encoder']['type'] = type_
#     token_characters_.append(copy)

base_cnn_token_characters = {
    "type": "character_encoding",
    "embedding": {
        "embedding_dim": 25
    },
    "encoder": {
        "type": "cnn",
        "embedding_dim": 25,
        "num_filters": 100,
        "ngram_filter_sizes": [5]
    },
    "dropout": 0.25,
}
cnn_token_characters_space = [[0.25]]
for dropout, in list(itertools.product(*cnn_token_characters_space)):
    copy = deepcopy(base_cnn_token_characters)
    copy['dropout'] = dropout
    token_characters_.append(copy)

encoders = []
base_encoder = {
    "type": "lstm",
    "input_size": 200,
    "hidden_size": 200,
    "num_layers": 3,
    "dropout": 0.25,
    "bidirectional": True
}
encoder_space = [[0.25], [2, 3, 4, 5], [600, 800, 1000], ['lstm']]
encoder_points = list(itertools.product(*encoder_space))
for dropout, num_layers, hidden_size, type_ in encoder_points:
    copy = deepcopy(base_encoder)
    copy['dropout'] = dropout
    copy['num_layers'] = num_layers
    copy['hidden_size'] = hidden_size
    copy['type'] = type_
    encoders.append(copy)

In [5]:
import itertools
import random
space = [[random.randint(0, 2**32) for i in range(32)]]
points = list(itertools.product(*space))
random.shuffle(points)
print('Number of points: %d' % len(points))

Number of points: 32


In [6]:
import torch
from lib.utils import config_logging
from lib.utils import new_experiment_folder
from allennlp.commands.train import train_model
import json
import copy
import shutil



config_logging()

for i, (rand_int,) in enumerate(points):
    hyperparameters = copy.deepcopy(crf_tagger)
    hyperparameters['pytorch_seed'] = rand_int
    hyperparameters['numpy_seed'] = rand_int
    hyperparameters['random_seed'] = rand_int
    print('–' * 100)
    experiment_folder = new_experiment_folder(label='subject_recognition_grid_search_' + str(i),
                                          parent_directory='../../experiments/')
    print('Seed: %s' % rand_int)
    print('Serialization Directory:', experiment_folder)
    params = Params(hyperparameters)
    train_model(params=params, serialization_dir=experiment_folder)
    shutil.rmtree(experiment_folder)
    params = None
    torch.cuda.empty_cache()

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
Seed: 3626764237
Serialization Directory: ../../experiments/subject_recognition_grid_search_0.02_11_19:16:28
[2018-02-11 19:16:28,991][MainProcess][allennlp.common.params][INFO] random_seed = 3626764237
[2018-02-11 19:16:28,991][MainProcess][allennlp.common.params][INFO] numpy_seed = 3626764237
[2018-02-11 19:16:28,992][MainProcess][allennlp.common.params][INFO] pytorch_seed = 3626764237
[2018-02-11 19:16:28,994][MainProcess][allennlp.common.checks][INFO] Pytorch version: 0.3.0.post4
[2018-02-11 19:16:28,997][MainProcess][allennlp.common.params][INFO] dataset_reader.type = sequence_tagging
[2018-02-11 19:16:28,998][MainProcess][allennlp.common.params][INFO] dataset_reader.token_indexers.tokens.type = single_id
[2018-02-11 19:16:28,999][MainProcess][allennlp.common.params][INFO] dataset_reader.token_indexers.tokens.namespace = tokens
[2018-02-11 19:16:29,000][MainProcess][allennlp.common

74520it [00:02, 30378.12it/s]

[2018-02-11 19:16:31,528][MainProcess][allennlp.common.params][INFO] validation_data_path = ./../../data/subject_recognition/dev.txt
[2018-02-11 19:16:31,528][MainProcess][allennlp.commands.train][INFO] Reading validation data from ./../../data/subject_recognition/dev.txt
[2018-02-11 19:16:31,529][MainProcess][allennlp.data.dataset_readers.sequence_tagging][INFO] Reading instances from lines in file at: ./../../data/subject_recognition/dev.txt



10648it [00:00, 53691.03it/s]

[2018-02-11 19:16:31,739][MainProcess][allennlp.common.params][INFO] test_data_path = None
[2018-02-11 19:16:31,740][MainProcess][allennlp.commands.train][INFO] Creating a vocabulary using train, validation data.
[2018-02-11 19:16:31,831][MainProcess][allennlp.common.params][INFO] vocabulary.directory_path = None


...

[2018-02-11 22:12:50,528][MainProcess][allennlp.training.trainer][INFO] Training


token_accuracy: 1.00, accuracy: 1.00, loss: 0.02 ||:   5%|5         | 60/1165 [00:04<01:32, 12.01it/s]

KeyboardInterrupt: 

token_accuracy: 1.00, accuracy: 1.00, loss: 0.02 ||:   5%|5         | 60/1165 [00:20<06:08,  3.00it/s]

[2018-02-11 22:13:34,463][MainProcess][allennlp.models.archival][INFO] archiving weights and vocabulary to ../../experiments/subject_recognition_grid_search_2.02_11_20:56:18/model.tar.gz
